# Spark tutorial

For the linux guys, load ipython notebook with:

```export PYSPARK_DRIVER_PYTHON="jupyter"
export PYSPARK_DRIVER_PYTHON_OPTS="notebook --pylab inline"
pyspark```

Two important variables : contexts

In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
sqlContext

Ipython notebook supports some shell commands.

In [3]:
!ls

mllib-tutorial.ipynb	       Spark_correction_tutorial.ipynb
mllib-tutorial-students.ipynb  Spark_tutorial_students.ipynb


## Warm up with RDDs

Let's compute [Pi](https://fr.wikipedia.org/wiki/Pi) using Monte Carlo Simulation.

![alt text](http://www.physics.smu.edu/fattarus/pi.png "Pi simulation")

In [27]:
n_points = 1000

points = sc.parallelize( range(n_points) )

In [29]:
from random import random

def generate_random_pt(_):
    x = random() * 2 - 1 # -> Rnd number between -1 and 1
    y = random() * 2 - 1
    return x, y

def is_inside_unary_circle((x, y)):
    return 1 if x ** 2 + y ** 2 <= 1 else 0

points = points.map(generate_random_pt)

inside_points = points.filter(is_inside_unary_circle)

In [16]:
print("Example point : {}".format(inside_points.first()))

Example point : (0.5878496590819144, 0.6292085841390693)


In [34]:
inside_area = inside_points.count()
overall_area=4
print( "Pi estimation is {}".format(overall_area*inside_area/float(n_points)) )

Pi estimation is 3.1532


In [37]:
n_points = 10000

points = sc.parallelize( range(n_points) )

points = points.map(generate_random_pt)

inside_points = points.filter(is_inside_unary_circle)

inside_area = inside_points.count()
print( "Pi estimation is {}".format(overall_area*inside_area/float(n_points)) )

Pi estimation is 3.1816


## Basic arithmetic
Basic operations on an RDD.

## Caching

Interesting experiments

* You can take advantage of common pipeline elements

In [38]:
n_trials=100
n_throws=10

pil_data = sc.parallelize(range(n_trials))

def generate_play():
    return "pile" if random()>0.5 else "face"

def generate_game(_):
    return [generate_play() for _ in range(n_throws)]

pil_data = pil_data.map(generate_game)

print(pil_data.first())

['face', 'face', 'pile', 'pile', 'face', 'pile', 'pile', 'face', 'pile', 'face']


In [41]:
print(pil_data.take(2))

[['face', 'face', 'pile', 'pile', 'face', 'pile', 'pile', 'face', 'pile', 'face'], ['face', 'pile', 'pile', 'face', 'pile', 'face', 'face', 'pile', 'pile', 'face']]


In [42]:
def firstg(games):
    return games[0]

def firstwog(games):
    return games[:2]

In [43]:
%%time
pil_data.map(firstg).filter(lambda res: res=="pile").count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 56.9 ms


50

In [44]:
%%time
pil_data.map(firstg).filter(lambda res: res=="face").count()

CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 77.7 ms


48

In [45]:
%%time
cached_rdd = pil_data.map(firstg)
cached_rdd.filter(lambda res: res=="pile").count()
cached_rdd.filter(lambda res: res=="face").count()

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 169 ms


In [48]:
import numpy as np

def count_res(games):
    return np.unique(games, return_counts=True)[1].tolist()

def get_face_result(counts):
    return counts[0]

def get_pile_result(counts):
    return counts[1]

In [51]:
%%time
pil_data.map(count_res).filter(lambda res: get_pile_result(res)==2 ).count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 105 ms


8

In [52]:
%%time
pil_data.map(count_res).filter(lambda res: get_face_result(res)==2 ).count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 73.8 ms


0

In [53]:
%%time
cached_rdd = pil_data.map(count_res)
cached_rdd.filter(lambda res: get_pile_result(res)==2).count()
cached_rdd.filter(lambda res: get_face_result(res)==2).count()

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 153 ms


Something interesting?...
Try to increase the size of the RDD and see the computing time...

#### Exercice

Try to count every occurences of the different possible tuples (pile, pile) (face, face) ... using a cache optimised process.

### Exercice: KMeans

Let's get to work ... Implement a k means algorithm on a dataset

Extract some features to decrease computing time

In [56]:
from random import choice

means = [0, 1, 5, -2]

def generate_random_pt(_):
    
    mean = choice(means)
    
    return np.random.randn() + mean

In [57]:
n_points = 10000

points = sc.parallelize( range(n_points) )

points = points.map(generate_random_pt)

In [61]:
points.first()

-0.3359149423833519

Useful function
Find the closest center in a list of centers to the datapoint: datapoint -> index of the closest centroid

In [62]:
import numpy as np

def closestTo(datapoint, centerList):
    
    # Compute distances
    
    return np.argmin(distanceList)

Random initialisation of the initial centroids

In [92]:
N = 10 # Number of centroids
from numpy.random import rand
centroidsList = []
for indCentroid in range(N):
    centroidsList.append(rand(1,11))

In [93]:
closestTo(random(), centroidsList)

8

Simplest kMeans algorithm:
* Compute centroids of each cluster
* Update centroids
* Repeat

Explanations:
* First transformation:
    - Compute centroid for each datapoint
    - datapoint -> Tuple ( closest centroid Index , ( datapoint, 1 ) )

* Reduce By Key: Aggregate for each centroid
    - centroid Index , ( sum of datapoints , number of datapoints )
    - ( datapoint1, pop1 ) and ( datapoint2, pop2 ) => (datapoint1 + datapoint2 , pop1+pop2)

* So that at the end, 
    - (cluster Index, (sum of datapoint in cluster , number of datapoints in cluster) )
    - Can compute the centroid

In [103]:
for t in range(5):
    
    # Compute centroids of clusters
    
    
    
    centroidRDD = points.map() # Compute centroids for each element of the RDD
    
    centroidStats = centroidRDD.reduceByKey().collect()
    
    # Update centroids with new ones
    
    for stats in newCentroids:
        centroidIndex = stats[0]
        centroidDataSum = stats[1][0]
        centroidDataCount = stats[1][0]
        newCentroid = centroidDataSum / centroidDataCount

        centroidsList[centroidIndex] = newCentroid
        
    # Repeat ...

Have you recovered the centroids? Can you plot it?